Click the play button in the cell below to load all tools we'll need. You'll be asked whether you want to use the Sandbox environment (for testing) or the Production environment (using 'live' data from Alma).

You'll then be asked for your list of MMSIDs - this should be a text file (.txt), with an MMSID on each line. If you copy a list of MMSIDs from excel, open notepad, and paste, this will give you exactly what you need.

In [7]:
import os
from IPython.display import display
import ipywidgets as widgets

def clicked_sand(arg):
    global apiKey
    apiKey = os.environ['API_KEY_SAND']
    with out:
        print('\rUsing Sandbox data   ', end='')

def clicked_prod(arg):
    global apiKey
    apiKey = os.environ['API_KEY_PROD']
    with out:
        print('\rUsing Production data', end='')

print('Would you like to use the Sandbox or Production environment?')
button_sand = widgets.Button(description = 'Sandbox', tooltip='Sandbox')   
button_sand.on_click(clicked_sand)

button_prod = widgets.Button(description = 'Production', tooltip='Production')   
button_prod.on_click(clicked_prod)

out = widgets.Output()
buttons = widgets.HBox([button_sand, button_prod])
outdisplay = widgets.VBox([out])
uploader = widgets.FileUpload(description ='Upload file', accept='.txt', multiple=False)
display(buttons, outdisplay)
print('Upload your MMSID txt file here - this should be a text file (.txt), with an MMSID on each line')
display(uploader)




Would you like to use the Sandbox or Production environment?


Upload your MMSID txt file here - this should be a text file (.txt), with an MMSID on each line


FileUpload(value=(), accept='.txt', description='Upload file')

In [5]:
one = widgets.Text(value='Hello World text', placeholder='Type something', description='described', disabled=False)
two = widgets.Textarea(value='Hello World textarea', placeholder='Type something', description='described', disabled=False)
three = widgets.Combobox(placeholder='Choose something', options=['A', 'B', 'C'], description='described', ensure_option=True, disabled=Fals
display(one)
display(two)
display(three)

l8xxb55ee13fda44401c8a7e13497e929adb
